# ETF Pipeline

> ETF flow decompositions pipeline.
- skip_exec: true

In [ ]:
# | hide
# | eval: false
%load_ext autoreload
%autoreload 2

In [ ]:
# | default_exp etf_pipeline

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
import os

import click
from arcticdb import Arctic, QueryBuilder
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
from pathlib import Path
from lobster_tools.config import MainConfig, Overrides, register_configs, get_config
from lobster_tools.preprocessing import *
from lobster_tools.querying import *
from lobster_tools.flow_decomposition import *
from lobster_tools.config import etf_to_equities
import pandas as pd
from sklearn.linear_model import LinearRegression
from itertools import product
import datetime
from dataclasses import dataclass
from functools import partial
import json
from typing import Literal
import numpy as np
from pprint import pprint

In [ ]:
# | eval: false
register_configs()
cfg = get_config(overrides=Overrides.full_server)

In [ ]:
# | eval: false
directory_path = cfg.data_config.csv_files_path
etfs = cfg.universe.etfs
equities = cfg.universe.equities
# fix this
date_range = tuple(cfg.data_config.date_range)
markouts = cfg.hyperparameters.markouts
finest_resample = cfg.hyperparameters.finest_resample
max_markout = cfg.hyperparameters.max_markout

load: Literal["both", "messages", "book"] = "both"
clip_trading_hours = True
add_ticker_column = True

ticker = "APA"
date_range = ("2020-01-02", "2020-01-04")
date_range = "2020-01-02"

In [ ]:
# | eval: false
data = Data(
    directory_path="/nfs/home/nicolasp/home/data/tmp",
    ticker="COP",
    date_range="2020-01-03",
    aggregate_duplicates=False,
)
lobster = Lobster(data=data)

yo


In [ ]:
# with initialize_config_module(version_base=None, config_module="lobster_tools.config"):
#     cfg = compose(overrides=["data_config=server"])
#     print(cfg)

### Single day

In [ ]:
# | eval: false
Data(directory_path=directory_path, ticker="APA", date_range="2020-01-02")

In [ ]:
# | eval: false
equity_data = Data(directory_path=directory_path,
                   ticker=ticker,
                   date_range=date_range,
                   load=load,
                   clip_trading_hours=clip_trading_hours,
                   add_ticker_column=add_ticker_column)

equity_lobsters = Lobster(equity_data)

### Multi-day

In [ ]:
# | eval: false
equity_data = [
    Data(
        directory_path=directory_path,
        ticker=ticker,
        date_range=date_range,
        load=load,
        clip_trading_hours=clip_trading_hours,
        add_ticker_column=add_ticker_column,
    )
    for ticker in equities
]

equity_lobsters = [Lobster(data) for data in equity_data]

equity_executions = pd.concat(
    [lobster.messages.pipe(get_executions) for lobster in equity_lobsters]
)
equity_executions.sort_index(inplace=True)

In [ ]:
# | eval: false
etf_data = [
    Data(
        directory_path=directory_path,
        ticker=ticker,
        date_range=date_range,
        load=load,
        clip_trading_hours=clip_trading_hours,
        add_ticker_column=add_ticker_column,
    )
    for ticker in etfs
]

etf_lobsters = [Lobster(data) for data in etf_data]

etf_executions = pd.concat(
    [lobster.messages.pipe(get_executions) for lobster in etf_lobsters]
)
etf_executions.sort_index(inplace=True)

### Load with ArcticDB

In [ ]:
# | eval: false
db_path = cfg.db.db_path

conn = f'lmdb://{db_path}'
arctic = Arctic(conn)
library = "NASDAQ"
arctic_library = arctic[library]
print('available tickers:')
print(arctic_library.list_symbols())

available tickers:
['SLG', 'CSCO', 'HPE', 'PWR', 'AVB', 'CNC', 'NOW', 'ROP', 'XYL', 'AMGN', 'PXD', 'TROW', 'CB', 'AON', 'LB', 'PNR', 'SBAC', 'BAC', 'AMAT', 'ETR', 'SHW', 'CMS', 'WFC', 'LEN', 'PVH', 'VRSN', 'ZTS', 'PFE', 'XRX', 'CFG', 'HIG', 'MMC', 'MTD', 'FE', 'MLM', 'ZBH', 'XEC', 'AZO', 'SWK', 'CBOE', 'CTAS', 'FTNT', 'NSC', 'ICE', 'MKC', 'PSX', 'ABT', 'CDW', 'EQIX', 'FTV', 'HON', 'GD', 'PSA', 'VNO', 'XEL', 'KHC', 'ORCL', 'DGX', 'DE', 'GPC', 'NVR', 'INFO', 'FLIR', 'EBAY', 'CE', 'DG', 'LYB', 'MAS', 'ALB', 'NWSA', 'DD', 'BK', 'DLTR', 'PFG', 'CVS', 'COST', 'CI', 'HOG', 'IRM', 'MAR', 'OMC', 'UAA', 'WCG', 'HBAN', 'LKQ', 'WRK', 'RMD', 'MA', 'PRGO', 'AAP', 'MET', 'MXIM', 'PEP', 'MSCI', 'ADBE', 'ITW', 'KSS', 'STT', 'FRC', 'XOM', 'HCA', 'PLD', 'DRI', 'GLW', 'SPGI', 'EFX', 'MCI', 'BR', 'JCI', 'COP', 'FISV', 'HBI', 'GIS', 'CPRT', 'HPQ', 'RCL', 'APH', 'DHI', 'LUV', 'ANTM', 'JWN', 'ABC', 'VZ', 'NTRS', 'VTR', 'RE', 'KLAC', 'TMO', 'NDAQ', 'NWL', 'LDOS', 'TWTR', 'MCD', 'AEE', 'NI', 'MU', 'TGT', 'FCX',

In [ ]:
# | eval: false
ticker = "UDR"
q = QueryBuilder()
q = q.date_range((pd.Timestamp(f"2016-01-04T09:29"), pd.Timestamp(f"2016-01-06T16:01")))
# q = q[q.event.isin(EventGroup.EXECUTIONS.value)]
df = arctic_library.read(ticker, query_builder=q, columns=["event", "order_id", "size", "price", "direction", "bid_price_1", "ask_price_1"]).data
df

NoSuchVersionException: E_NO_SUCH_VERSION read_dataframe_version: version matching query 'latest' not found for symbol 'UDR'

In [ ]:
# | eval: false
ticker = "APA"
# q = QueryBuilder()
# q = q.date_range((pd.Timestamp(f"2020-01-07T09:29"), pd.Timestamp(f"2020-01-10T16:01")))
# q = q[q.event.isin(EventGroup.EXECUTIONS.value)]
df = library.read(ticker, columns=["event", "order_id", "size", "price", "direction", "bid_price_1", "ask_price_1"]).data
df.tail()

,event,order_id,size,price,direction,ask_price_1,bid_price_1
datetime,,,,,,,
2020-01-10 15:59:59.752373153,3,275468889,100,32.39,-1,32.38,32.35
2020-01-10 15:59:59.924996854,3,275930645,700,32.32,1,32.38,32.35
2020-01-10 15:59:59.924996854,1,276591509,700,32.33,1,32.38,32.35
2020-01-10 15:59:59.952231315,3,276507025,200,32.38,-1,32.38,32.35
2020-01-10 15:59:59.952248660,3,276507897,100,32.38,-1,32.38,32.35


In [ ]:
# | eval: false
def get_data_from_arctic(ticker, date):
    q = QueryBuilder()
    q = q.date_range((pd.Timestamp(f"{date}T09:29"), pd.Timestamp(f"{date}T16:01")))
    # get executions
    q = q[q.event.isin(EventGroup.EXECUTIONS.value)]
    df = library.read(ticker, query_builder=q, columns=["event", "order_id", "size", "price", "direction", "bid_price_1", "ask_price_1"]).data
    
    # do assign here for now
    df = df.assign(ticker=ticker)
    return df

In [ ]:
# | eval: false
def get_data_from_arctic_(ticker):
    q = QueryBuilder()
    q = q.date_range((pd.Timestamp(f"2020-01-02T09:29"), pd.Timestamp(f"2020-01-03T16:01")))
    # get executions
    q = q[q.event.isin(EventGroup.EXECUTIONS.value)]
    df = library.read(ticker, query_builder=q, columns=["event", "order_id", "size", "price", "direction", "bid_price_1", "ask_price_1"]).data
    
    # do assign here for now
    df = df.assign(ticker=ticker)
    return df

In [ ]:
# | eval: false
# equities=["APA", "BKR", "HAL"]
# equities=['APA', 'XOM', 'EOG', 'OXY', 'MPC', 'HES', 'BKR', 'KMI', 'PXD', 'PSX', 'HAL', 'DVN', 'OKE', 'CVX', 'FANG', 'VLO', 'WMB', 'COP', 'SLB']
equities = [
    "HES",
    "EOG",
    "VLO",
    "DVN",
    "OXY",
    "PXD",
    "XOM",
    "COP",
    "WMB",
    "HAL",
    "PSX",
    "CVX",
    "OKE",
    "BKR",
]
etfs = ["XLE"]

In [ ]:
# | eval: false
equity_executions = pd.concat(
    [get_data_from_arctic_(ticker=ticker) for ticker in equities]
).sort_index()

In [ ]:
# | eval: false
equity_executions

In [ ]:
# | eval: false
etf_executions = pd.concat(
    [get_data_from_arctic(ticker=ticker, date="2020-01-02") for ticker in etfs]
).sort_index()

In [ ]:
# | eval: false
etf_executions = pd.concat(
    [get_data_from_arctic_(ticker=ticker) for ticker in etfs]
).sort_index()

In [ ]:
#TODO: pickle and load pickles.. looks like nbdev doesn't work with cell magics

In [ ]:
# | eval: false
%store etf_executions
%store equity_executions

### Cell magic doesn't work with nbdev(?)

In [ ]:
# | eval: false
%store -r

In [ ]:
# | eval: false
ofi_all = ofi(etf_executions, resample_freq="5T", suffix="all")
ofi_all

In [ ]:
# | eval: false
markout_times = markout_returns(ofi_all, markouts=markouts)
markout_times

In [ ]:
# | eval: false
mids = etf_executions.eval("bid_price_1 + (ask_price_1 - bid_price_1) / 2").resample(finest_resample, label="right").last()
mids = pd.DataFrame(mids, columns=etfs)
# mids.resample(resample_freq, label="right").last()

In [ ]:
# | eval: false
mids

In [ ]:
# | eval: false
# markout_times = markout_returns(ofi_all, markouts=markouts)
# mids = [resample_mid(lobster.book, resample_freq=finest_resample).rename(lobster.data.ticker) for lobster in etf_lobsters]
# mids = pd.concat(mids, axis=1)
# mids

In [ ]:
# | eval: false
def compute_returns():
    index = clip_for_markout(etf_executions.resample(resample_freq, label="right").last(), max_markout=max_markout).index

    returns = {}
    for ticker in etfs:
        df = pd.DataFrame(index=index)
        print(df)
        for markout in ["0S"] + markouts:
            df[f"_{markout}"] = mids.loc[df.index + pd.Timedelta(markout), ticker].values

        for markout in markouts:
            df.eval(f"return_{markout} = (_{markout} / _0S ) - 1", inplace=True)

        df["return_contemp"] = mids[ticker].resample("5T").first().pct_change()
        df_returns = df.filter(regex="return")
        df_returns.columns = [column.replace("return_", "") for column in df_returns.columns]
        df_returns.columns = [("_" + column if column[0].isdigit() else column) for column in df_returns.columns ]
        # new addition
        df_returns.fillna(0, inplace=True)
        returns[ticker] = df_returns
    return returns

returns = compute_returns()
returns[etfs[0]]

In [ ]:
# | eval: false
returns[etfs[0]].sample(10)

In [ ]:
# | eval: false
def regression_table(
    X: pd.DataFrame,  # covariates. in this case, OFI for a single ETF
    y: pd.DataFrame,  # response variable. in this case, mid to mid returns at various markouts
):
    """Compute X.len * y.len univariate regressions. For each column in X, regress against each one column of y."""
    X, y = restrict_common_index(X, y)
    
    regression_results = []

    for x_col_name, y_col_name in product(X.columns, y.columns):
        x_col = X[x_col_name].values.reshape(-1, 1)
        y_col = y[y_col_name].values
        
        model = LinearRegression(fit_intercept=False)
        model.fit(x_col, y_col)

        intercept = model.intercept_
        coefficient = model.coef_[0]
        r2 = model.score(x_col, y_col)

        regression_results.append(
            {
                "id": x_col_name + "_" + y_col_name,
                "intercept": intercept,
                "coefficient": coefficient,
                "r2": r2,
            }
        )
        
    regression_results_df = pd.DataFrame(regression_results)
    regression_results_df.set_index("id", inplace=True)
    return regression_results_df

regression_table(ofi_all, returns[etfs[0]])

In [ ]:
# | eval: false
drop_all_neighbor_cols(etf_executions)

In [ ]:
# | eval: false
etf_executions_neighbors = add_neighbors(etf_executions=etf_executions, equity_executions=equity_executions, tolerance=tolerances)
etf_executions_neighbors

In [ ]:
# | eval: false
def compute_neighbor_statistics(etf_executions_neighbors: pd.DataFrame):
    neighbor_statistics = etf_executions_neighbors.filter(regex="^_").notna().sum() / len(etf_executions_neighbors)
    return neighbor_statistics

neighbor_statistics = compute_neighbor_statistics(etf_executions_neighbors)
# pd.DataFrame({'has_neighbor':neighbor_statistics}).style.format("{:.2%}")
pd.DataFrame({'has_neighbor':neighbor_statistics})

In [ ]:
# | eval: false
etf_executions_features = append_features(etf_executions=etf_executions_neighbors, equity_executions=equity_executions)

In [ ]:
# | eval: false
etf_executions_features["_500us_num_trades"] = etf_executions_features._500us_num_trades_os_af + etf_executions_features._500us_num_trades_os_bf + etf_executions_features._500us_num_trades_ss_af + etf_executions_features._500us_num_trades_ss_bf

In [ ]:
# | eval: false
etf_executions_features._500us_num_trades.hist()

In [ ]:
# | eval: false
val_counts = etf_executions_features.value_counts(subset="_500us_num_trades")
val_counts.where(val_counts > 100).dropna().index.values.tolist()

In [ ]:
# | eval: false
ENOUGH_DATA = 7
etf_executions_features.query(f"_500us_num_trades <= {ENOUGH_DATA}")

CLOSE_PRICE = ( etf_executions_features.query("direction == 1").iloc[-1].price + etf_executions_features.query("direction == -1").iloc[-1].price ) / 2
print(CLOSE_PRICE)

In [ ]:
# | eval: false
etf_executions_features_no_auc = etf_executions_features[etf_executions_features.event.isin([4,5])]
etf_executions_features_no_auc = etf_executions_features_no_auc.query(f"_500us_num_trades <= {ENOUGH_DATA}").assign(pnl_to_close = lambda df_: ( CLOSE_PRICE - df_.price ) / df_.price * 1e4 * df_.direction)


In [ ]:
# | eval: false
etf_executions_features_no_auc = etf_executions_features_no_auc.assign(hit_ratio = lambda df_: ((df_.pnl_to_close) > 0 ))
summary_statistics = etf_executions_features_no_auc.groupby(by="_500us_num_trades").agg(ppt_mean=("pnl_to_close","mean"),ppt_std=("pnl_to_close","std"), hit_raio=("hit_ratio","mean"))

In [ ]:
# | eval: false
import matplotlib.pyplot as plt
for col in summary_statistics:
    summary_statistics[col].plot(title=col, kind="bar")
    plt.show()

### Multi-day

In [ ]:
# | eval: false
df = etf_executions_features_no_auc

In [ ]:
# | eval: false
closing_prices = df.resample('D').transform('last').price

In [ ]:
# | eval: false
df["closing_price"] = closing_prices

In [ ]:
# | eval: false
df.eval("pnl_to_close = (closing_price - price) * 1e4 * direction", inplace=True)
df.eval("hit_ratio = (pnl_to_close > 0)", inplace=True)

In [ ]:
# | eval: false
summary_statistics = df.groupby(by="_500us_num_trades").agg(ppt_mean=("pnl_to_close","mean"),ppt_std=("pnl_to_close","std"), hit_ratio=("hit_ratio","mean"))

In [ ]:
# | eval: false
import matplotlib.pyplot as plt
for col in summary_statistics:
    summary_statistics[col].plot(title=col, kind="bar")
    plt.show()

In [ ]:
# | eval: false
%store etf_executions_neighbors
%store etf_executions_features

In [ ]:
# etf_executions_features = marginalise(etf_executions_features, over='same_sign/opposite_sign')

In [ ]:
# etf_executions_features.columns

In [ ]:
# etf_executions_features["_500us_num_trades"] = etf_executions_features._500us_num_trades_os_af + etf_executions_features._500us_num_trades_os_bf + etf_executions_features._500us_num_trades_ss_af + etf_executions_features._500us_num_trades_ss_bf

In [ ]:
# etf_execution_features.filter(regex="^_").hist()

In [ ]:
# easy way without query builder
# date_range = [datetime.date(year=2020, month=1, day=2), datetime.date(year=2020, month=1, day=3)]
# df = library.read(ticker, date_range=date_range).data

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()